In this project, we will calculate a lead-field matrix that can convert brain activity in the AAL2 atlas to a 64 electrode EEG cap.
A lead-field matrix is a matrix `M` s.th.
`y_m = M_mn x_n`
where `y_m` is the EEG activity at electrode `m` and `x_n` is the neural activity at voxel `n`. Note: I will call the elements of a brain that can generate activity a "voxel", but they could also be represented as "nodes" in a mesh representation in Field trip.
These are the steps:
- First, we calculate a high-resolution lead-field matrix using Field Trip. Field trip converts the neural activity that is represented as dipoles on each voxel of the brain into the electric field at the electrodes. These dipoles generate an electric field that is simulated until it reaches the EEG electrodes.
- Typically, this problem is done in reverse: you start with EEG data `y` and want to end up with brain activity `x`. We have the inverse problem: we want to know what `y` is, if *ALL* voxels `x` are active.
- Electric fields are linear that means we can compute the *overall* lead field matrix that measure the effect of all voxels from the *individual* lead field matrices that measure only the effect of a single voxel.
- I can imagine there are two ways of doing this: either there is a way in field trip to "activate" the entire brain at once and get the *overall* lead field matrix (probably uses some kind of data structure to hold this), or we would have to go through the brain one voxel `n` (example: `n=1`) at a time, activate that voxel and measure the lead field matrix `M_m1` (it's actually just a column of the overall matrix) for that voxel. We do that `N` times such that we get `N` lead field matrices that we can just sum up to get the overall one: `M_mn = M_m1 + M_m2 + ... + M_mN`
- The first way of doing this would be a lot more efficient.
- We then get a high-res lead field matrix (the *overall* matrix) with `N` columns.
- Export this matrix into a python-usable format (simple numpy array). We can work with this matrix later in Python.
- Now we want to create a low-res lead field matrix defined over the AAL2 atlas.
- For this, we need to summarize the columns of the high-res matrix such that values that belong to the same AAL2 brain region are reduced into one column.
- We can take a column of this matrix that corresponds to voxels that belong to the same brain area and simply take the average EEG activity that this brain area causes by simply taking the average of these columns and using it as one.
- Now we should have a low-dimensional lead field matrix with `n` being 120 (or: 80 only cortical areas, excluding subcortical).
# Testing results
- Now that we have the lead field matrix, we can do cool things with neurolib.
- We can simulate brain activity `x` and calculate EEG activity `y` by simply multiplying the activity with the matrix.
- We now have EEG activity that we can compare with the original activity `x` and see which effects this introduces. First thing to check would be to simulate a brain that produces slow oscillations and compare the power spectra of `y` and `x`.
- Same for a brain that is oscillating at higher frequencies.
- The neurolib whole-brain model parameters to simulate slow oscillations or high-frequency oscillations will be shared with you.
The end.

Matlab Script: https://www.fieldtriptoolbox.org/workshop/ohbm2018/forward/ 

Forward Model: https://www.fieldtriptoolbox.org/development/module/forward/

In [2]:
import numpy as np
import scipy as sp
import matplotlib as plt